In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import json

from PIL import Image
import torchvision
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from sklearn.metrics import *
import time
from datetime import datetime
import os
from torch.utils import data
import random
import copy
import itertools
import io
import uuid
from sklearn.model_selection import KFold, train_test_split
import itertools
import warnings
warnings.filterwarnings('ignore')

import wandb
wandb_username = 'denizjafari'
local_username = 'denizjafari'

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda:1') 
else:
    device = torch.device('cpu')
print(device)

cuda:1


In [3]:
# root directory
root_dir = "/home/andreasabo/Documents/HNProject/"
split_file_base = "/home/andreasabo/Documents/HNUltra/"

# data directory on current machine: abhishekmoturu, andreasabo, denizjafari, navidkorhani
data_dir = "/home/" + local_username + "/Documents/HNProject/all_label_img/"

# read target df
csv_path = os.path.join(root_dir, "all_splits_1000000.csv")
data_df = pd.read_csv(csv_path, usecols=['subj_id', 'image_ids', 'view_label', 'view_train'])

# Are we doing the final test?
test_data = True

### **Reading Data Indicies and Labels**

In [4]:
label_mapping = {'Other':0, 'Saggital_Right':1, 'Transverse_Right':2, 
                 'Saggital_Left':3, 'Transverse_Left':4, 'Bladder':5}
label_unmapping = {0: 'Other', 1:'Saggital_Right', 2: 'Transverse_Right', 
                   3:'Saggital_Left', 4:'Transverse_Left', 5: 'Bladder'}

data_df['view_label'] = data_df['view_label'].map(label_mapping)

train_df = data_df[data_df.view_train == 1]
test_df = data_df[data_df.view_train == 0]

unique_subj = train_df.subj_id.unique()

# Create the splits for 5-fold cross validation based on subj_id
data_split_file = split_file_base + 'data_splits.json'
# just load from file
print("Reading splits from file")
with open(data_split_file, 'r') as f:
    all_folds = json.load(f)

# If we're testing, overwrite the training data with the entire train/test data
if test_data:
    train_images = train_df.image_ids.tolist()
    test_images = test_df.image_ids.tolist()
    train_labels = train_df.view_label.tolist()
    test_labels = test_df.view_label.tolist()

    cur_fold = {'train_ids': train_images, 'test_ids': test_images, 'train_labels': train_labels, 'test_labels': test_labels}

    
    all_folds['test'] = cur_fold

Reading splits from file


In [5]:
print(len(all_folds['test']['train_ids']))
print(len(all_folds['0']['train_ids']))

print(len(all_folds['test']['test_ids']))
print(len(all_folds['0']['valid_ids']))

13958
11081
5070
2877


In [6]:
# MODEL DIRECTORIES 


vae50_dir = '/home/navidkorhani/Documents/HNProject/HNUltra/saved models/vae_model_h800_l50.pt'
vae100_dir = "/home/navidkorhani/Documents/HNProject/HNUltra/results/h800_l100_e30/vae_model.pt"
vae200_dir = "/home/navidkorhani/Documents/HNProject/HNUltra/results/h800_l200_e30/vae_model.pt"


In [7]:
# source: https://scikit-learn.org/0.16/auto_examples/model_selection/plot_confusion_matrix.html#example-model-selection-plot-confusion-matrix-py
def plot_confusion_matrix(cm,rep, classnames, title='Confusion matrix', cmap=plt.cm.Blues):
    fig, ax = plt.subplots(1, figsize=(14, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classnames))
    plt.xticks(tick_marks, classnames, rotation=45)
    plt.yticks(tick_marks, classnames)
    
    
    thresh = cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black", fontsize=19)
        
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig('ViewNetConfMtrx' + str(rep) + '.png')

# Models Initialization 

In [8]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [9]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 256

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224


    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == 'vae50':
        
        model_ft = VAE_50()
        input_size = 256
        
    elif model_name == 'vae100':
        
        model_ft = VAE_100()
        input_size = 256
        
    elif model_name == 'vae200':
        
        model_ft = VAE_200()
        input_size = 256
        
        
    elif model_name == 'viewnet':
        conv1_filters = 8
        conv2_filters = 16
        conv3_filters = 32
        linear1_size = 512
        dropout = 0.25
        model_ft = ViewNet(num_classes, conv1_filters, conv2_filters, conv3_filters, linear1_size, dropout)
        input_size = 256
        
    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size




# Models Architectures 

In [10]:
# custom view labeller 
class ViewNet(nn.Module):

    def __init__(self, num_classes, conv1_filters, conv2_filters, conv3_filters, linear1_size, dropout):
        super(ViewNet, self).__init__()
        self.conv1_filters = conv1_filters
        self.conv2_filters = conv2_filters
        self.conv3_filters = conv3_filters
        self.linear1_size = linear1_size
        self.drop_percent = dropout
        self.max_pool = 4
        self.conv_output = int(self.conv3_filters*(256/(self.max_pool**3))*(256/(self.max_pool**3)))
        print("conv_output: ", self.conv_output)

        
        self.conv1 = nn.Conv2d(1, self.conv1_filters, 4, padding=2)
        self.conv2 = nn.Conv2d(self.conv1_filters, self.conv2_filters, 4, padding=2)
        self.conv3 = nn.Conv2d(self.conv2_filters, self.conv3_filters, 4, padding=2)
        self.pool = nn.MaxPool2d(self.max_pool, self.max_pool)
        self.dropout = nn.Dropout(self.drop_percent)
        self.linear1 = nn.Linear(self.conv_output, self.linear1_size)
        self.linear2 = nn.Linear(self.linear1_size, num_classes)
    
    def forward(self, x):
        x = self.pool(self.dropout(F.relu(self.conv1(x))))
        x = self.pool(self.dropout(F.relu(self.conv2(x))))
        x = self.pool(self.dropout(F.relu(self.conv3(x))))
        x = x.view(-1, self.conv_output) 
        x = self.dropout(F.relu((self.linear1(x))))
        x = self.linear2(x)
        return x
    
    
class VAE_50(nn.Module):
    def __init__(self):
        super(VAE_50, self).__init__()
        
        hidden_dim = 800
        latent_dim = 50
        self.fc1 = nn.Linear(65536, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, latent_dim)
        self.fc22 = nn.Linear(hidden_dim, latent_dim)
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, 65536)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        #print("z.size() =", z.size())
        h3 = F.relu(self.fc3(z))
        #print("h3.size() =", h3.size())
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 65536))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar
    
    
class VAE_100(nn.Module):
    def __init__(self):
        super(VAE_100, self).__init__()
        
        hidden_dim = 800
        latent_dim = 100
        self.fc1 = nn.Linear(65536, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, latent_dim)
        self.fc22 = nn.Linear(hidden_dim, latent_dim)
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, 65536)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        #print("z.size() =", z.size())
        h3 = F.relu(self.fc3(z))
        #print("h3.size() =", h3.size())
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 65536))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar
    
class VAE_200(nn.Module):
    def __init__(self):
        super(VAE_200, self).__init__()
        
        hidden_dim = 800
        latent_dim = 200
        self.fc1 = nn.Linear(65536, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, latent_dim)
        self.fc22 = nn.Linear(hidden_dim, latent_dim)
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, 65536)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        #print("z.size() =", z.size())
        h3 = F.relu(self.fc3(z))
        #print("h3.size() =", h3.size())
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 65536))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [11]:
# Code from: https://gist.github.com/stefanonardo/693d96ceb2f531fa05db530f3e21517d
class EarlyStopping(object):
    def __init__(self, mode='min', min_delta=0, patience=10, percentage=True):
        self.mode = mode
        self.min_delta = min_delta
        self.patience = patience
        self.best = None
        self.num_bad_epochs = 0
        self.is_better = None
        self._init_is_better(mode, min_delta, percentage)

        if patience == 0:
            self.is_better = lambda a, b: True
            self.step = lambda a: False

    def step(self, metrics):
        if self.best is None:
            self.best = metrics
            return False

        if np.isnan(metrics):
            return True

        if self.is_better(metrics, self.best):
            self.num_bad_epochs = 0
            self.best = metrics
        else:
            self.num_bad_epochs += 1

        if self.num_bad_epochs >= self.patience:
            return True

        return False

    def _init_is_better(self, mode, min_delta, percentage):
        if mode not in {'min', 'max'}:
            raise ValueError('mode ' + mode + ' is unknown!')
        if not percentage:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - min_delta
            if mode == 'max':
                self.is_better = lambda a, best: a > best + min_delta
        else:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - (
                            best * min_delta / 100)
            if mode == 'max':
                self.is_better = lambda a, best: a > best + (
                            best * min_delta / 100)

# Model Training 

In [12]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25,rep=5, is_inception=False, final_testing=True):
    es = EarlyStopping(patience = 15)
    stop_now = 0

    since = time.time()
    classnames = ['Other', 'Saggital_Right', 'Transverse_Right', 'Saggital_Left','Transverse_Left', 'Bladder']
    #val_acc_history = []
    
    #val_metrics_list = []
    #train_metrics_list = []
    y_test_last = []
    y_pred_last = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    epoch_with_best_val_acc = num_epochs -1
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 54)

        if stop_now:
            break
        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            
            running_preds = []
            running_labels = []

            # save the needed data 
            new_col_ids = []
            #new_col_labels = []
            #new_col_preds = []
            
            # Iterate over data.
            for inputs, labels, ids in dataloaders[phase]:
                labels = labels.type(torch.long)
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                if epoch == (num_epochs -1):
                    
                   
                    new_col_ids += np.array(ids).tolist()
                    
                    
                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        labels = torch.argmax(labels, 1)
                        running_preds += torch.argmax(outputs, 1).tolist()
                        running_labels += labels.tolist()
                        loss = criterion(outputs, labels)

                    preds = torch.argmax(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            print('{} loss:\t{:.4f} | {} acc:\t{:.4f}\n'.format(phase, epoch_loss, phase, epoch_acc))

            if phase == 'train':
                wandb.log({'epoch': epoch, 'train_acc':epoch_acc, 'train_loss':epoch_loss})
                
                cur_train_metrics = {}
                                # compute and log f1, precision, recall for each class
                for c in range(6):
                    running_labels = np.asarray(running_labels)
                    running_preds = np.asarray(running_preds)

                    cur_c_labs_bin = np.asarray([0] *len(running_labels))
                    cur_c_preds_bin = np.asarray([0] *len(running_labels))

                    # Need to binarize
                    cur_c_preds_bin[running_preds == c] = 1
                    cur_c_labs_bin[running_labels == c] = 1
                    f1 = f1_score(cur_c_labs_bin, cur_c_preds_bin)
                    precision = precision_score(cur_c_labs_bin, cur_c_preds_bin)
                    recall = recall_score(cur_c_labs_bin, cur_c_preds_bin)
                    
                    cur_train_metrics['train_' + label_unmapping[c] + '_f1'] = f1
                    cur_train_metrics['train_' + label_unmapping[c] + '_precision'] = precision
                    cur_train_metrics['train_' + label_unmapping[c] + '_recall'] = recall
                    
                
                #train_metrics_list.append(cur_train_metrics)
                
                average_types = ['macro', 'micro', 'weighted']
                average_metrics_to_log = ['precision', 'recall', 'f1score', 'support']
                average_dict = {'epoch': epoch}
                for av in average_types:
                    results_tuple = precision_recall_fscore_support(running_labels, running_preds, average=av)
                    for m in range(len(average_metrics_to_log)):      
                        average_dict[phase + '_'+ average_metrics_to_log[m] +'_average_' + av] = results_tuple[m]
                        cur_train_metrics[phase + '_'+ average_metrics_to_log[m] +'_average_' + av] = results_tuple[m]
                cur_train_metrics[phase + '_acc_average'] = accuracy_score(running_labels, running_preds)                  
                average_dict[phase + '_acc_average'] = accuracy_score(running_labels, running_preds)     
                wandb.log(cur_train_metrics)
                
            if phase == 'test':
                wandb.log({'test_loss':epoch_loss, 'test_acc':epoch_acc, 'epoch': epoch})
                if epoch == (num_epochs-1):
            
                    cur_val_metrics = {}
                    # compute and log f1, precision, recall for each class
                    for c in range(6):
                        running_labels = np.asarray(running_labels)
                        running_preds = np.asarray(running_preds)
                        new_col_ids = np.asarray(new_col_ids)
                        # confusion matrix of the last epoch
                        conf_mx = confusion_matrix(running_labels, running_preds)

                        cur_c_labs_bin = np.asarray([0] *len(running_labels))
                        cur_c_preds_bin = np.asarray([0] *len(running_labels))

                        # Need to binarize
                        cur_c_preds_bin[running_preds == c] = 1
                        cur_c_labs_bin[running_labels == c] = 1
                        f1 = f1_score(cur_c_labs_bin, cur_c_preds_bin)
                        precision = precision_score(cur_c_labs_bin, cur_c_preds_bin)
                        recall = recall_score(cur_c_labs_bin, cur_c_preds_bin)

                        wandb.log({'test_' + label_unmapping[c] + '_f1': f1})
                        wandb.log({'test_' + label_unmapping[c] + '_precision': precision})
                        wandb.log({'test_' + label_unmapping[c] + '_recall': recall})
                        #wandb.log({'test_' + '_confusion_matrix': conf_mx})

                        cur_val_metrics['val_' + label_unmapping[c] + '_f1'] = f1
                        cur_val_metrics['val_' + label_unmapping[c] + '_precision'] = precision
                        cur_val_metrics['val_' + label_unmapping[c] + '_recall'] = recall

                    average_types = ['macro', 'micro', 'weighted']
                    average_metrics_to_log = ['precision', 'recall', 'f1score']
                    average_dict = {'epoch': epoch}
                    for av in average_types:
                        results_tuple = precision_recall_fscore_support(running_labels, running_preds, average=av)
                        for m in range(len(average_metrics_to_log)):      
                            average_dict[phase + '_'+ average_metrics_to_log[m] +'_average_' + av] = results_tuple[m]
                            cur_val_metrics[phase + '_'+ average_metrics_to_log[m] +'_average_' + av] = results_tuple[m]
                    cur_val_metrics[phase + '_acc_average'] = accuracy_score(running_labels, running_preds)                  
                    average_dict[phase + '_acc_average'] = accuracy_score(running_labels, running_preds)     
                    print(cur_val_metrics)
                    wandb.log(cur_val_metrics)


                    #val_metrics_list.append(cur_val_metrics)
                
            if phase == 'train':
                print(classification_report(running_labels, running_preds))
                train_acc = epoch_acc
                if epoch == (num_epochs-1):
                    best_model_wts = copy.deepcopy(model.state_dict())
                    torch.save(model.state_dict(), os.path.join(wandb.run.dir, "ViewNetFinalTrainModel.pt"))
                
            if phase == 'test' and epoch == (num_epochs-1):
                best_acc = epoch_acc
                best_acc_train = train_acc
                print(classification_report(running_labels, running_preds))
                print()
                #print(conf_mx)
                plot_confusion_matrix(conf_mx,rep,classnames, title='Test Data Confusion matrix - ViewNet', 
                                      cmap=plt.cm.Blues)
                
           # if phase == 'test' and epoch_acc > best_acc:
            #    best_acc = epoch_acc
            #    best_acc_train = train_acc
            #    epoch_with_best_val_acc = epoch
            #    best_model_wts = copy.deepcopy(model.state_dict())
            #    torch.save(model.state_dict(), os.path.join(wandb.run.dir, "model.pt"))
            #    print(classification_report(running_labels, running_preds))

            #if phase == 'test':
            #    val_acc_history.append(epoch_acc)
            #    if es.step(epoch_loss) and not final_testing:
            #        stop_now = 1
            #        print("EARLY STOPPING " + str(epoch))
            #        break

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val acc: {:4f}\n'.format(best_acc))
    
    # Directly save the best results in this fold
    wandb.config.best_acc = best_acc
    wandb.config.best_epoch = epoch_with_best_val_acc

    #wandb.config.val_acc_history = val_acc_history
    wandb.config.best_epoch = epoch_with_best_val_acc
    
    #wandb.config.update(val_metrics_list[epoch_with_best_val_acc])
    wandb.config.update(cur_val_metrics)
    #wandb.config.update(train_metrics_list[epoch_with_best_val_acc])
    wandb.config.update(cur_train_metrics)
    
    metrics_from_best_epoch = {'best_epoch': epoch_with_best_val_acc, 'last_epoch': epoch}
    metrics_from_best_epoch.update( cur_val_metrics )
    #metrics_from_best_epoch.update( train_metrics_list[epoch_with_best_val_acc] )
    metrics_from_best_epoch.update(cur_train_metrics)
    metrics_from_best_epoch.update( {'val_acc': best_acc.cpu(), 'train_acc': best_acc_train.cpu()} )    
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    #return model, val_acc_history, metrics_from_best_epoch
    return model, metrics_from_best_epoch, new_col_ids, running_labels, running_preds
    

In [13]:
def train5fold(network_configs,criterion_used,  model_ft, lr, wd, amsgrad, repetitions):
    fold = 'test'

    random_str = str(uuid.uuid4()).split("-")[0]
    best_metrics_per_fold = []
    final_df = pd.DataFrame()
    model_base = copy.deepcopy(model_ft)
    
    for i in range(repetitions):
        now = datetime.now()
        date_time = now.strftime("%d-%m-%Y.%H:%M:%S")
        wandb.init(project='hnultra_test', entity=wandb_username, name=local_username + '_rep_' + str(i), group=random_str)
        partition = all_folds[fold]

        model_ft = copy.deepcopy(model_base)
        model_ft = model_ft.to(device)
        wandb.watch(model_ft)

        # Gather the parameters to be optimized/updated in this run. If we are
        #  finetuning we will be updating all parameters. However, if we are
        #  doing feature extract method, we will only update the parameters
        #  that we have just initialized, i.e. the parameters with requires_grad
        #  is True.
        params_to_update = model_ft.parameters()
        #print("Params to learn:")
        if feature_extract:
            params_to_update = []
            for name,param in model_ft.named_parameters():
                if param.requires_grad == True:
                    params_to_update.append(param)
                    print("\t",name)
        else:
            for name,param in model_ft.named_parameters():
                if param.requires_grad == True:
                    print("\t",name)

        # Observe that all parameters are being optimized
        optimizer_ft = optim.Adam(params_to_update, lr=lr, weight_decay=wd, amsgrad=amsgrad)

        # Setup the loss fxn
        criterion = criterion_used

        shuffle = True
        num_workers = 0
        params = {'batch_size': batch_size,
                  'shuffle': shuffle,
                  'num_workers': num_workers}

        config_dict = {'i': i, 'batch_size': batch_size, 'shuffle': shuffle, 'num_workers': num_workers, 'fold': fold,
                       'lr': lr, 'wd': wd, 'amsgrad': amsgrad, 'model_name': model_name,'criterion': criterion, 'num_classes': num_classes, 
                       'num_epochs': num_epochs, 'feature_extract': feature_extract, "pretrain": pretrain }

        wandb.config.update(config_dict)
        wandb.config.update(network_configs)
        # Tranforms
        trans = transforms.Compose([transforms.RandomAffine(degrees=8, translate=(0.1, 0.1), scale=(0.95,1.25))])

        # Generators
        training_set = Dataset(partition['train_ids'], partition['train_labels'], transformations=trans)
        training_generator = data.DataLoader(training_set, **params)

        validation_set = Dataset(partition['test_ids'], partition['test_labels'])
        validation_generator = data.DataLoader(validation_set, **params)

        dataloaders_dict = {'train':training_generator, 'test':validation_generator}

        # Train & Evaluate
        model_ft, metrics_from_best_epoch, rep_ids, rep_labels, rep_preds = train_model(model_ft, 
                    dataloaders_dict, criterion, optimizer_ft, num_epochs,i, is_inception=(model_name=="inception"))
        best_metrics_per_fold.append(metrics_from_best_epoch)
     
        final_df['IDs_iter_' + str(i)] = np.array(rep_ids)
        final_df['labels_iter_' + str(i)] = np.array(rep_labels)
        final_df['preds_iter_' + str(i)] = np.array(rep_preds)
       

    # Calculate the performance metrics on the best model in each fold
    wandb.init(project='hnultra', entity=wandb_username, name=local_username + '_ALL', group=random_str)
    config_dict['fold'] = -1
    config_dict['i'] = -1
    wandb.config.update(config_dict)
    wandb.config.update(network_configs)

    
    metrics_all = {}
    for fold in best_metrics_per_fold:
        for key in fold:
            if key not in metrics_all:
                metrics_all[key] = [fold[key]]
            else:
                metrics_all[key].append(fold[key]) 
                
  

    metrics_to_log = {}
    for m in metrics_all:
        try:
            
            metric_list = np.asarray(metrics_all[m])
    #         print(m)
    #         print(metric_list)
    #         print(type(metric_list))
            metrics_to_log[m + '_mean'] = metric_list.mean()    
            metrics_to_log[m + '_stdev'] = metric_list.std()
        except: 
            pass
    wandb.config.update(metrics_to_log)
    
    return final_df
  

In [14]:
class Dataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, list_IDs, labels, transformations=None):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs
        self.transformations = transformations
        
  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        img_path = data_dir + ID + '.jpg'
        image = Image.open(img_path).convert('L')
        
        if self.transformations:
            image = self.transformations(image)
        
        image = ToTensor()(image)
        
        y = torch.FloatTensor([0]*6)        
        y[int(self.labels[index])] = 1

        return image, y, ID

In [15]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception, viewnet]
model_name = "viewnet"

# Number of classes in the dataset: right_sag, right_trav, left_sag, left_trav, bladder, other
num_classes = 6

# Batch size for training (change depending on how much memory you have)
batch_size = 100

# Number of epochs to train for
num_epochs = 46

# Flag for feature extracting. When False, we finetune the whole model; when True we only update the reshaped layer params
feature_extract = False

# Flag for whether or not to use pretrained model
pretrain = False

criterion_used = nn.CrossEntropyLoss()

In [ ]:
repetitions = 5

conv1_filters = 8
conv2_filters = 16
conv3_filters = 32
linear1_size = 512

dropout = 0.25
lr = 0.0005
wd = 0.001
amsgrad = False

#config_string = f"{conv1_filters}_{conv2_filters}_{conv3_filters}_{linear1_size}_{dropout}_{lr}_{wd}_{amsgrad}"
model_ft = ViewNet(num_classes, conv1_filters, conv2_filters, conv3_filters, linear1_size, dropout)
run_configs = {'lr': lr, 'wd': wd, 'amsgrad': amsgrad,'dropout': dropout, 
              'conv1_filters': conv1_filters, 'conv2_filters': conv2_filters, 
              'conv3_filters': conv3_filters, 'linear1_size': linear1_size }

final_df = train5fold(run_configs,criterion_used, model_ft, lr, wd, amsgrad, repetitions)

conv_output:  512


wandb: Wandb version 0.8.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


	 conv1.weight
	 conv1.bias
	 conv2.weight
	 conv2.bias
	 conv3.weight
	 conv3.bias
	 linear1.weight
	 linear1.bias
	 linear2.weight
	 linear2.bias
Epoch 1/46
------------------------------------------------------
train loss:	1.5552 | train acc:	0.3935

              precision    recall  f1-score   support

           0       0.39      0.90      0.54      4943
           1       0.37      0.10      0.15      2225
           2       0.16      0.01      0.01      1539
           3       0.37      0.19      0.25      2284
           4       0.31      0.00      0.01      1669
           5       0.61      0.30      0.40      1298

    accuracy                           0.39     13958
   macro avg       0.37      0.25      0.23     13958
weighted avg       0.37      0.39      0.30     13958

test loss:	1.4932 | test acc:	0.4613

Epoch 2/46
------------------------------------------------------
train loss:	1.3770 | train acc:	0.4592

              precision    recall  f1-score   support

    

In [ ]:
final_df.head()

In [ ]:
final_df.to_csv (r'final_viewnet_data.csv', index = False, header=True)